In [74]:
from pathlib import Path
import pandas as pd
import numpy as np
import polars as pl
import duckdb
import os
import pandera as pa
import io

pd.set_option('display.max_columns', 35)
pd.set_option("display.max_colwidth",40)
pd.set_option("display.max_rows", 500)

FROM_EXPERIMENT_CSV_IN = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_02_EXP_SELECT_CIK_CSV")
TO_EXPERIMENT_PARQUET = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_03_EXP_SELECT_CIK_PARQUET")

REDUCED_SELECT_CSV =Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_02_EXP_SELECT_CIK_CSV_REDUCED")

#----
processed_tables_copy = Path(r"E:\app_data\dropbox_13f_files\processed_tables\processed_tables_copy")
TR_00_ALL_CSV_REDUCED = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_00_ALL_CSV_REDUCED")

### Transforming original `csv` files into new `csv` with reduced number of columns
* new folder with reduced `csv`: 
  1. From `processed_tables_copy =Path(r"E:\app_data\dropbox_13f_files\processed_tables\processed_tables_copy")`
  2. To ```TR_00_ALL_CSV_REDUCED = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_00_ALL_CSV_REDUCED")```
  
* It's a one off code


In [ ]:
%%time

columns = ['cik', 'cusip8', 'cusip9','value', 'shares','rdate', 'fdate',\
           'address', 'form', 'shrsOrPrnAmt', 'putCall', 'nameOfIssuer', 'titleOfClass', 'type', 'dsource']

dtypes = {'cusip8': str, 'cusip9': str , 'titleOfClass': str, 'form': str,
          'putCall': str, 'shrsOrPrnAmt': str, 'value': pl.Float64, 'shares': pl.Float64, 
          'nameOfIssuer': str, 'cik' : pl.Int64, 'address': str, 'type': str,'num5': str,
          'deviation':str, 'shrout':str,'num3': str,'num2': str, 'num6':str,'num7': str,'num4': str,
           'votingAuthority': str, 'in_universe': str,'prc': str, 'split': str,
           'investmentDiscretion': str, 'rdate': str, 'fdate': str, 'dsource': str}

pd_dtypes = {'cusip8': str, 'cusip9': str , 'titleOfClass': str, 'form': 'category', 'putCall': 'category',
           'shrsOrPrnAmt': 'category', 'value': 'float64', 'shares': 'float64', 'type': 'category', 'nameOfIssuer': str,
           'cik' : 'int64', 'address': 'category',  'dsource': 'category'}

# pd_dtypes_validation = {'cusip8': str, 'cusip9': str , 'titleOfClass': str, 'form': 'category', 'putCall': 'category',
#            'shrsOrPrnAmt': 'category', 'value': 'Int64', 'shares': 'Int64', 'type': 'category', 'nameOfIssuer': str,
#            'cik' : 'int64', 'address': 'category',  'dsource': 'category'}


# for file in processed_tables_copy.rglob("*.csv"):
#     new_file_name = file.parts[-2]+"-"+file.name
#     new_file_path = Path(os.path.join(TR_00_ALL_CSV_REDUCED, new_file_name))
#     if new_file_path.exists():  continue

    
#     schema = pl.scan_csv(file).schema
#     read_cols = list(set(schema.keys()).intersection(columns))
#     df = pl.read_csv(file, columns=read_cols, dtypes=dtypes)

#     for col in columns:
#         if col not in df.columns:
#             df = df.with_column(pl.lit(None, dtype=dtypes[col]).alias(col))

#     df = df.with_columns([pl.col("rdate").str.strptime(pl.Date, fmt="%Y%m%d"),
#                         pl.col("fdate").str.strptime(pl.Date, fmt="%Y%m%d"),
#                         pl.col("cusip8").str.to_uppercase(),
#                         pl.col("cusip9").str.to_uppercase(),
#                         pl.lit('dropbox').alias('dsource')]).select(columns)  

#     df.write_csv(new_file_path, sep=",")
#     df = df.to_pandas().astype(pd_dtypes_validation)

In [32]:
# len(list(processed_tables_copy.rglob("*.csv"))), len(list(TR_00_ALL_CSV_REDUCED.glob("*.csv")))
# (308585, 308585)

### Working on selected **reduced csv** and **txt** files to manually improve the data quality with `pandera`

In [314]:
selected_cik = [2230,3520,5272,7195,7789,9015,10742,14661,16972,18349,18748,19475,19617,21175,22657,24386,35442,35527,36066,36104,36644,36966,38777,39263,40417,40545,44365,45319,49205,50863,51762,51812,51964,52234,53417,59558,59951,60086,61227,67698,70858,71210,71259,72971,73124,80255,84616,89014,92230,93751,98758,102212,102909,105495,108572,200217,201772,216851,276101,310051,312348,313028,313807,314949,314957,314984,315014,315032,315038,315054,315066,315080,315157,315297,315498,316011,318989,320335,320376,351051,351173,351262,354204,356264,700529,704051,707179,712537,713676,714142,720672,723204,728083,728100,728618,732905,733020,740272,740913,741073,743127,750641,754811,757657,759944,762152,763212,763848,764068,764106,764112,764529,764532,765443,769317,769954,769963,775368,776867,778963,779519,788714,790354,790502,791191,791490,796848,799003,799004,801051,806097,807249,807985,808722,809339,809443,810265,810384,810386,810672,810716,811360,811454,813917,813933,814133,814375,816788,819535,820027,820123,820124,820289,820478,820743,821197,822581,823621,825293,829407,831001,831571,836372,837592,842782,842941,846222,846633,846788,846797,850401,850529,852743,854157,857508,859872,860486,860561,860580,860585,860643,860644,860645,860662,860748,860828,860857,861176,861177,861462,861787,862469,866361,866842,868491,869178,869179,869353,869367,872080,872163,872259,872573,872732,873630,874791,877134,877338,878228,881432,883511,883677,883782,883790,883803,883961,883965,884300,884314,884414,884423,884541,884546,884548,884566,884589,885062,885415,886982,887402,887777,887818,889232,891287,891478,893738,894205,894300,894309,895213,895421,897070,897378,897599,898358,898382,898399,898413,899211,900169,900529,900973,902219,902367,902464,902584,903064,903944,903947,903949,905567,905591,905608,906304,908195,909151,909661,911274,912938,914933,914976,915287,915325,916542,917579,918893,919079,919185,919192,919458,919489,919497,919530,919538,919859,920440,920441,921531,921669,922127,922439,922898,922940,923093,923116,923469,924166,924171,924181,926688,926833,926834,928047,928196,928566,928568,928633,930441,931097,932024,932974,933429,934639,934999,936698,936753,936936,936941,936944,937394,937522,937589,937615,937760,937886,938076,938206,938487,938582,938592,938759,939219,940445,941560,943719,944234,944804,945625,945631,947822,947996,948518,948669,949012,949509,949615,949623,949853,1000097,1000742,1002152,1002672,1002784,1004244,1005354,1005607,1005817,1006364,1006378,1006407,1006435,1007280,1007399,1007524,1008322,1008877,1008894,1008895,1008929,1008937,1009003,1009005,1009012,1009016,1009022,1009076,1009207,1009209,1009232,1009254,1009258,1009262,1010873,1010911,1011443,1011659,1013234,1013536,1013538,1013701,1014306,1014315,1014736,1014738,1015079,1015083,1015086,1015308,1016150,1016287,1016683,1016972,1017115,1017645,1017918,1018331,1018674,1018825,1019231,1020066,1020317,1020580,1020585,1020617,1020918,1021008,1021117,1021223,1021249,1021258,1021642,1021926,1023279,1024716,1025421,1026200,1026710,1027451,1027796,1027817,1029160,1030618,1030815,1031972,1032814,1033225,1033427,1033475,1033505,1033974,1033984,1034184,1034196,1034524,1034541,1034546,1034549,1034642,1034771,1034886,1035350,1035463,1035912,1036248,1036325,1037389,1037558,1037763,1037792,1038661,1039565,1039807,1040190,1040197,1040198,1040210,1040273,1040592,1040762,1041241,1041885,1042046,1044207,1044797,1044905,1044924,1044929,1044936,1046187,1047339,1048921,1049648,1049650,1050442,1050463,1050470,1051359,1052100,1053013,1053054,1053055,1054074,1054425,1054522,1054554,1054677,1055290,1055544,1055963,1055964,1055966,1056053,1056288,1056466,1056488,1056491,1056515,1056516,1056527,1056549,1056559,1056581,1056593,1056807,1056821,1056825,1056827,1056831,1056859,1056958,1056973,1057395,1057439,1058022,1058470,1058800,1059187,1061186,1061768,1062938,1065349,1065350,1066816,1067324,1067926,1067983,1068829,1070134,1071483,1072843,1074027,1074034,1074266,1074273,1076598,1077148,1077583,1078013,1078246,1078658,1078841,1079112,1079114,1079736,1079738,1079930,1080071,1080107,1080117,1080132,1080166,1080171,1080173,1080197,1080201,1080351,1080374,1080380,1080381,1080382,1080386,1080493,1080523,1080628,1080818,1081019,1081198,1082020,1082215,1082327,1082339,1082461,1082491,1082509,1082621,1082917,1083323,1083340,1084207,1084208,1084683,1085041,1085163,1085227,1085601,1085936,1086477,1086483,1086611,1086619,1086762,1086763,1088859,1088875,1088950,1089707,1089755,1089911,1089991,1090413,1091561,1091860,1091923,1092203,1092290,1092351,1092903,1093276,1093589,1094584,1094749,1095836,1096783,1097218,1097278,1097833,1100710,1101250,1102062,1102578,1102598,1103245,1103738,1103804,1103882,1103887,1104186,1104329,1104366,1105468,1105471,1105497,1105837,1105863,1105909,1106129,1106191,1106500,1106505,1106832,1107261,1107310,1108893,1108965,1108969,1109147,1110806,1113629,1114618,1114739,1114928,1115941,1116247,1125727,1125816,1129770,1133219,1134152,1140334,1140771,1142031,1142062,1158583,1389426,1398739,1469219]
len(selected_cik)

676

In [322]:
# copy raw .txt filings for the selected 676 ciks (active in 1999 and 2000) to a separate folder `TR_00_TEST_676_CIK_TXT`
selected_cik = [2230,3520,5272,7195,7789,9015,10742,14661,16972,18349,18748,19475,19617,21175,22657,24386,35442,35527,36066,36104,36644,36966,38777,39263,40417,40545,44365,45319,49205,50863,51762,51812,51964,52234,53417,59558,59951,60086,61227,67698,70858,71210,71259,72971,73124,80255,84616,89014,92230,93751,98758,102212,102909,105495,108572,200217,201772,216851,276101,310051,312348,313028,313807,314949,314957,314984,315014,315032,315038,315054,315066,315080,315157,315297,315498,316011,318989,320335,320376,351051,351173,351262,354204,356264,700529,704051,707179,712537,713676,714142,720672,723204,728083,728100,728618,732905,733020,740272,740913,741073,743127,750641,754811,757657,759944,762152,763212,763848,764068,764106,764112,764529,764532,765443,769317,769954,769963,775368,776867,778963,779519,788714,790354,790502,791191,791490,796848,799003,799004,801051,806097,807249,807985,808722,809339,809443,810265,810384,810386,810672,810716,811360,811454,813917,813933,814133,814375,816788,819535,820027,820123,820124,820289,820478,820743,821197,822581,823621,825293,829407,831001,831571,836372,837592,842782,842941,846222,846633,846788,846797,850401,850529,852743,854157,857508,859872,860486,860561,860580,860585,860643,860644,860645,860662,860748,860828,860857,861176,861177,861462,861787,862469,866361,866842,868491,869178,869179,869353,869367,872080,872163,872259,872573,872732,873630,874791,877134,877338,878228,881432,883511,883677,883782,883790,883803,883961,883965,884300,884314,884414,884423,884541,884546,884548,884566,884589,885062,885415,886982,887402,887777,887818,889232,891287,891478,893738,894205,894300,894309,895213,895421,897070,897378,897599,898358,898382,898399,898413,899211,900169,900529,900973,902219,902367,902464,902584,903064,903944,903947,903949,905567,905591,905608,906304,908195,909151,909661,911274,912938,914933,914976,915287,915325,916542,917579,918893,919079,919185,919192,919458,919489,919497,919530,919538,919859,920440,920441,921531,921669,922127,922439,922898,922940,923093,923116,923469,924166,924171,924181,926688,926833,926834,928047,928196,928566,928568,928633,930441,931097,932024,932974,933429,934639,934999,936698,936753,936936,936941,936944,937394,937522,937589,937615,937760,937886,938076,938206,938487,938582,938592,938759,939219,940445,941560,943719,944234,944804,945625,945631,947822,947996,948518,948669,949012,949509,949615,949623,949853,1000097,1000742,1002152,1002672,1002784,1004244,1005354,1005607,1005817,1006364,1006378,1006407,1006435,1007280,1007399,1007524,1008322,1008877,1008894,1008895,1008929,1008937,1009003,1009005,1009012,1009016,1009022,1009076,1009207,1009209,1009232,1009254,1009258,1009262,1010873,1010911,1011443,1011659,1013234,1013536,1013538,1013701,1014306,1014315,1014736,1014738,1015079,1015083,1015086,1015308,1016150,1016287,1016683,1016972,1017115,1017645,1017918,1018331,1018674,1018825,1019231,1020066,1020317,1020580,1020585,1020617,1020918,1021008,1021117,1021223,1021249,1021258,1021642,1021926,1023279,1024716,1025421,1026200,1026710,1027451,1027796,1027817,1029160,1030618,1030815,1031972,1032814,1033225,1033427,1033475,1033505,1033974,1033984,1034184,1034196,1034524,1034541,1034546,1034549,1034642,1034771,1034886,1035350,1035463,1035912,1036248,1036325,1037389,1037558,1037763,1037792,1038661,1039565,1039807,1040190,1040197,1040198,1040210,1040273,1040592,1040762,1041241,1041885,1042046,1044207,1044797,1044905,1044924,1044929,1044936,1046187,1047339,1048921,1049648,1049650,1050442,1050463,1050470,1051359,1052100,1053013,1053054,1053055,1054074,1054425,1054522,1054554,1054677,1055290,1055544,1055963,1055964,1055966,1056053,1056288,1056466,1056488,1056491,1056515,1056516,1056527,1056549,1056559,1056581,1056593,1056807,1056821,1056825,1056827,1056831,1056859,1056958,1056973,1057395,1057439,1058022,1058470,1058800,1059187,1061186,1061768,1062938,1065349,1065350,1066816,1067324,1067926,1067983,1068829,1070134,1071483,1072843,1074027,1074034,1074266,1074273,1076598,1077148,1077583,1078013,1078246,1078658,1078841,1079112,1079114,1079736,1079738,1079930,1080071,1080107,1080117,1080132,1080166,1080171,1080173,1080197,1080201,1080351,1080374,1080380,1080381,1080382,1080386,1080493,1080523,1080628,1080818,1081019,1081198,1082020,1082215,1082327,1082339,1082461,1082491,1082509,1082621,1082917,1083323,1083340,1084207,1084208,1084683,1085041,1085163,1085227,1085601,1085936,1086477,1086483,1086611,1086619,1086762,1086763,1088859,1088875,1088950,1089707,1089755,1089911,1089991,1090413,1091561,1091860,1091923,1092203,1092290,1092351,1092903,1093276,1093589,1094584,1094749,1095836,1096783,1097218,1097278,1097833,1100710,1101250,1102062,1102578,1102598,1103245,1103738,1103804,1103882,1103887,1104186,1104329,1104366,1105468,1105471,1105497,1105837,1105863,1105909,1106129,1106191,1106500,1106505,1106832,1107261,1107310,1108893,1108965,1108969,1109147,1110806,1113629,1114618,1114739,1114928,1115941,1116247,1125727,1125816,1129770,1133219,1134152,1140334,1140771,1142031,1142062,1158583,1389426,1398739,1469219]

from pathlib import Path
import shutil

# Define the paths
# main_directory = Path('E:\\app_data\\sec_apps_data\\speed_test\\filings_13f_full')
# filings_directory = main_directory / 'filings'
# selected_filings_directory = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_00_TEST_676_CIK_TXT")

# # Create the selected_filings_directory if it doesn't exist
# selected_filings_directory.mkdir(parents=True, exist_ok=True)

# # Iterate through the subdirectories in the filings directory
# for subdir in filings_directory.iterdir():
#     if int(subdir.name) in selected_cik:
#         # Copy the subdirectory to the selected_filings_directory
#         shutil.copytree(subdir, selected_filings_directory / subdir.name)

In [10]:
# copy reduced `csv` filings for the selected 676 ciks (active in 1999 and 2000) to a separate folder `TR_01_TEST_676_CIK_CSV_REDUCED`
selected_cik = [2230,3520,5272,7195,7789,9015,10742,14661,16972,18349,18748,19475,19617,21175,22657,24386,35442,35527,36066,36104,36644,36966,38777,39263,40417,40545,44365,45319,49205,50863,51762,51812,51964,52234,53417,59558,59951,60086,61227,67698,70858,71210,71259,72971,73124,80255,84616,89014,92230,93751,98758,102212,102909,105495,108572,200217,201772,216851,276101,310051,312348,313028,313807,314949,314957,314984,315014,315032,315038,315054,315066,315080,315157,315297,315498,316011,318989,320335,320376,351051,351173,351262,354204,356264,700529,704051,707179,712537,713676,714142,720672,723204,728083,728100,728618,732905,733020,740272,740913,741073,743127,750641,754811,757657,759944,762152,763212,763848,764068,764106,764112,764529,764532,765443,769317,769954,769963,775368,776867,778963,779519,788714,790354,790502,791191,791490,796848,799003,799004,801051,806097,807249,807985,808722,809339,809443,810265,810384,810386,810672,810716,811360,811454,813917,813933,814133,814375,816788,819535,820027,820123,820124,820289,820478,820743,821197,822581,823621,825293,829407,831001,831571,836372,837592,842782,842941,846222,846633,846788,846797,850401,850529,852743,854157,857508,859872,860486,860561,860580,860585,860643,860644,860645,860662,860748,860828,860857,861176,861177,861462,861787,862469,866361,866842,868491,869178,869179,869353,869367,872080,872163,872259,872573,872732,873630,874791,877134,877338,878228,881432,883511,883677,883782,883790,883803,883961,883965,884300,884314,884414,884423,884541,884546,884548,884566,884589,885062,885415,886982,887402,887777,887818,889232,891287,891478,893738,894205,894300,894309,895213,895421,897070,897378,897599,898358,898382,898399,898413,899211,900169,900529,900973,902219,902367,902464,902584,903064,903944,903947,903949,905567,905591,905608,906304,908195,909151,909661,911274,912938,914933,914976,915287,915325,916542,917579,918893,919079,919185,919192,919458,919489,919497,919530,919538,919859,920440,920441,921531,921669,922127,922439,922898,922940,923093,923116,923469,924166,924171,924181,926688,926833,926834,928047,928196,928566,928568,928633,930441,931097,932024,932974,933429,934639,934999,936698,936753,936936,936941,936944,937394,937522,937589,937615,937760,937886,938076,938206,938487,938582,938592,938759,939219,940445,941560,943719,944234,944804,945625,945631,947822,947996,948518,948669,949012,949509,949615,949623,949853,1000097,1000742,1002152,1002672,1002784,1004244,1005354,1005607,1005817,1006364,1006378,1006407,1006435,1007280,1007399,1007524,1008322,1008877,1008894,1008895,1008929,1008937,1009003,1009005,1009012,1009016,1009022,1009076,1009207,1009209,1009232,1009254,1009258,1009262,1010873,1010911,1011443,1011659,1013234,1013536,1013538,1013701,1014306,1014315,1014736,1014738,1015079,1015083,1015086,1015308,1016150,1016287,1016683,1016972,1017115,1017645,1017918,1018331,1018674,1018825,1019231,1020066,1020317,1020580,1020585,1020617,1020918,1021008,1021117,1021223,1021249,1021258,1021642,1021926,1023279,1024716,1025421,1026200,1026710,1027451,1027796,1027817,1029160,1030618,1030815,1031972,1032814,1033225,1033427,1033475,1033505,1033974,1033984,1034184,1034196,1034524,1034541,1034546,1034549,1034642,1034771,1034886,1035350,1035463,1035912,1036248,1036325,1037389,1037558,1037763,1037792,1038661,1039565,1039807,1040190,1040197,1040198,1040210,1040273,1040592,1040762,1041241,1041885,1042046,1044207,1044797,1044905,1044924,1044929,1044936,1046187,1047339,1048921,1049648,1049650,1050442,1050463,1050470,1051359,1052100,1053013,1053054,1053055,1054074,1054425,1054522,1054554,1054677,1055290,1055544,1055963,1055964,1055966,1056053,1056288,1056466,1056488,1056491,1056515,1056516,1056527,1056549,1056559,1056581,1056593,1056807,1056821,1056825,1056827,1056831,1056859,1056958,1056973,1057395,1057439,1058022,1058470,1058800,1059187,1061186,1061768,1062938,1065349,1065350,1066816,1067324,1067926,1067983,1068829,1070134,1071483,1072843,1074027,1074034,1074266,1074273,1076598,1077148,1077583,1078013,1078246,1078658,1078841,1079112,1079114,1079736,1079738,1079930,1080071,1080107,1080117,1080132,1080166,1080171,1080173,1080197,1080201,1080351,1080374,1080380,1080381,1080382,1080386,1080493,1080523,1080628,1080818,1081019,1081198,1082020,1082215,1082327,1082339,1082461,1082491,1082509,1082621,1082917,1083323,1083340,1084207,1084208,1084683,1085041,1085163,1085227,1085601,1085936,1086477,1086483,1086611,1086619,1086762,1086763,1088859,1088875,1088950,1089707,1089755,1089911,1089991,1090413,1091561,1091860,1091923,1092203,1092290,1092351,1092903,1093276,1093589,1094584,1094749,1095836,1096783,1097218,1097278,1097833,1100710,1101250,1102062,1102578,1102598,1103245,1103738,1103804,1103882,1103887,1104186,1104329,1104366,1105468,1105471,1105497,1105837,1105863,1105909,1106129,1106191,1106500,1106505,1106832,1107261,1107310,1108893,1108965,1108969,1109147,1110806,1113629,1114618,1114739,1114928,1115941,1116247,1125727,1125816,1129770,1133219,1134152,1140334,1140771,1142031,1142062,1158583,1389426,1398739,1469219]

from pathlib import Path
import shutil

# Define the paths
TR_00_ALL_CSV_REDUCED = Path(r'E:\app_data\dropbox_13f_files\processed_tables\TR_00_ALL_CSV_REDUCED')
TR_01_TEST_676_CIK_CSV_REDUCED = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_01_TEST_676_CIK_CSV_REDUCED")

# Create the selected_filings_directory if it doesn't exist
TR_01_TEST_676_CIK_CSV_REDUCED.mkdir(parents=True, exist_ok=True)

# # Iterate through the subdirectories in the filings directory
# for file in TR_00_ALL_CSV_REDUCED.glob("*.csv"):
#     file.name
#     # if subdir.name.isdigit() and int(subdir.name) in selected_cik:
#     if int(subdir.name) in selected_cik:
        # Copy the subdirectory to the selected_filings_directory
        # shutil.copytree(subdir, selected_filings_directory / subdir.name)

array([8.087139263148185, 125, -1, 125], dtype=object)

In [36]:
%%time
TR_01_TEST_676_CIK_CSV_REDUCED = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_01_TEST_676_CIK_CSV_REDUCED")

columns = ['cik', 'cusip8', 'cusip9','value', 'shares','rdate', 'fdate',\
           'address', 'form', 'shrsOrPrnAmt', 'putCall', 'nameOfIssuer', 'titleOfClass', 'type', 'dsource']



pd_dtypes = {'cusip8': str, 'cusip9': str , 'titleOfClass': str, 'form': 'category', 'putCall': 'category',
           'shrsOrPrnAmt': 'category', 'value': 'float64', 'shares': 'float64', 'type': 'category', 'nameOfIssuer': str,
           'cik' : 'int64', 'address': 'category',  'dsource': 'category'}

pd_dtypes_validation = {'cusip8': str, 'cusip9': str , 'titleOfClass': str, 'form': 'category', 'putCall': 'category',
           'shrsOrPrnAmt': 'category', 'value': 'int64', 'shares': 'int64', 'type': 'category', 'nameOfIssuer': str,
           'cik' : 'int64', 'address': 'category',  'dsource': 'category'}

dfs = []
for file in TR_02_EXP_SELECT_CIK_CSV_REDUCED.glob("2230-*.csv"):
    df = pl.read_csv(file, parse_dates=True) 
    
    dfs.append(df)
    
    df = pl.concat(dfs).to_pandas().astype(pd_dtypes_validation)
    # df = df.to_pandas().astype(pd_dtypes_validation)

NameError: name 'TR_02_EXP_SELECT_CIK_CSV_REDUCED' is not defined

In [97]:
# df.info()

In [45]:
df.query('index.isin([24, 80])').head(7)

# 0     26       1957109
# 1    183     718154107
# 2   1369     297659104
# 3   4145     362320103
# 4   4358      16962105
# 5   4373      67543101
# 6   4383      90078109

,cik,cusip8,cusip9,value,shares,rdate,fdate,address,form,shrsOrPrnAmt,putCall,nameOfIssuer,titleOfClass,type,dsource
24,2230,03475V10,03475V101,120,6100,2019-06-30,2019-07-29,2230/0001144204-19-036410.txt,13F-HR,SH,NaN,"AngioDynamics, Inc.",COM,xml,dropbox
80,2230,29251M10,29251M106,84,1000,2019-06-30,2019-07-29,2230/0001144204-19-036410.txt,13F-HR,SH,NaN,"Enanta Pharmaceuticals, Inc.",COM,xml,dropbox


In [49]:

basic_types_schema = pa.DataFrameSchema({
    "cik": pa.Column('int64'),
    "cusip8": pa.Column(str),
    "cusip9": pa.Column(str),
    "rdate": pa.Column("datetime64"),
    "fdate": pa.Column("datetime64"),
    "value": pa.Column("int64")
    
    
    })

In [ ]:
basic_types_schema.validate(df[columns])

In [ ]:
%%time
# Pandera also allows validating value ranges for numerical columns
# value_range_schema = pa.DataFrameSchema({
#     "LotArea": pa.Column(int, pa.Check(lambda s: s <= 1000000), nullable=False),
#     "YearBuilt": pa.Column(int, pa.Check.in_range(1800, 2022)),
# })

value_range_schema = pa.DataFrameSchema({
    "cik": pa.Column('int64'),
    "cusip8": pa.Column(str, pa.Check(lambda s: s.str.len() >= 8), pa.Check(lambda s: s.str.len() <= 9)),
    "cusip9": pa.Column(str, pa.Check(lambda s: s.str.len() >= 9), pa.Check(lambda s: s.str.len() <= 9)),

    # "cusip9": pa.Column(str, pa.Check(lambda s: ~s.str.contains("\+|\-"))),
    # "rdate": pa.Column("datetime64"),
    # "fdate": pa.Column("datetime64"),
    # "value": pa.Column("int64", pa.Check(lambda s: s <= 1000000), nullable=False)
                        })
%time value_range_schema.validate(df[columns])



In [58]:
filter = df.cusip9.str.contains('\+')
df.loc[filter]

,cik,cusip8,cusip9,value,shares,rdate,fdate,address,form,shrsOrPrnAmt,putCall,nameOfIssuer,titleOfClass,type,dsource


In [96]:
df.cusip9.sort_values(ascending=True)

8750    000361105
8751    001055102
8328    001055102
8539    001055102
9182    001055102
          ...    
8538    Y1771G102
8749    Y1771G102
8962    Y1771G102
6770    Y1771G102
9401    Y1771G102
Name: cusip9, Length: 9402, dtype: object

In [14]:
list(filings_676_txt.rglob("*.txt"))[0]

WindowsPath('E:/app_data/dropbox_13f_files/processed_tables/TR_00_TEST_676_CIK_TXT/1000097/0000919574-00-000478.txt')

In [176]:
%%time
import re
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# Define the regular expression to extrtact `cusip` values
exp_cusip = r"\b[A-Za-z0-9]{9}(?<![A-Za-z]{9})\b"

# Compile the regular expression
pattern_cusip = re.compile(exp_cusip)

# Open the text file
filings_676_txt = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_00_TEST_676_CIK_TXT")
# files = filings_676_txt.rglob("*.txt")  
files = filings_676_txt.rglob("*1006435/0001006435-12-000004.txt")  # -03-000131
big_df = pd.DataFrame()
for file in files:
    with open(file, 'r') as f:
        # Read the contents of the file
        text = f.read()
        # print(file)
        
    xml_pattern = r'<?xml'
    match_xml = re.search(xml_pattern, text, re.DOTALL | re.IGNORECASE)
    if match_xml: continue
    
    header_pattern = r'^.*<TABLE>'
    match_header = re.search(header_pattern, text, re.DOTALL | re.IGNORECASE)
    
    data_dict = dict()
    data_dict['rdate'] = datetime.strptime(
        re.compile(r"(?<=CONFORMED PERIOD OF REPORT:).*")
        .search(text)
        .group(0)
        .strip(),
        "%Y%m%d",
    ).date()
    
    data_dict['fdate'] = datetime.strptime(
        re.compile(r"(?<=FILED AS OF DATE:).*")
        .search(text)
        .group(0)
        .strip(),
        "%Y%m%d",
    ).date()
    
    accession = (
        re.compile(r"(?<=ACCESSION NUMBER:).*")
        .search(text)
        .group(0)
        .strip()+".txt"
    )
    cik = (
        re.compile(r"(?<=CENTRAL INDEX KEY:).*")
        .search(text)
        .group(0)
        .strip().lstrip('0')
        )
    data_dict['address']  = '/'.join([cik, accession])
    
    ## !!! todo !!! problem with cases where there is no lines for entry or value total at all. code break at the missing .group(0)
    
    entry_total = re.compile(r"(Entry Total(:)?).*").search(text)
    if entry_total:
        entry_total = entry_total.group(0).strip()
        entry_total = re.sub(r'\D', '', entry_total)
        if entry_total:
            entry_total = int(entry_total)
        else:
            entry_total = np.nan
    else:
        entry_total = np.nan
    data_dict['entry_total'] = entry_total
    
    #######----
    
    value_total = re.compile(r"(Value Total(:)?).*").search(text)
    if value_total:
        value_total = value_total.group(0).strip()
        value_total = re.sub(r'\D', '', value_total)
        if value_total:
            value_total = int(value_total)
        else:
            value_total = np.nan
            
    else:
        value_total = np.nan
    data_dict['value_total'] = value_total

    
    header_df = pd.DataFrame.from_dict([data_dict]).astype({'entry_total': 'Int64', 'value_total': 'Int64'})       
    
    # Search for table section
    table_pattern = r'<TABLE>.*$'
    # match_table = re.search(table_pattern, text, re.DOTALL | re.IGNORECASE)
    # if match_table:
        
    matches = pattern_cusip.finditer(text)
        # Create a list of the matched strings
    matched_strings = [match.group(0) for match in matches]
    matched_strings = set(matched_strings)
    matched_strings = set([s for s in matched_strings if not re.search(r'([^\d]{3,})', s)])

    df = pd.DataFrame(matched_strings, columns=['cusip'])
    # else:
    #     df = pd.DataFrame("no_cusip", columns=['cusip'])
        
    df = pd.concat([header_df, df], axis=1).ffill()
    big_df = pd.concat([big_df, df])

    # Filter out the strings that contain only non-digit characters
    # filtered_strings = set([s for s in matched_strings if any(c.isdigit() for c in s) and not all(c.isalpha() for c in s)])


CPU times: total: 203 ms
Wall time: 206 ms


In [ ]:
df

In [ ]:
# big_df = big_df.drop_duplicates() # before  # (15354045, 6)
# pd_dtypes = {'cusip': 'category' , 'value_total': 'Int64', 'entry_total': 'Int64', 'address': 'category',
#            'rdate': 'datetime64', 'fdate': 'datetime64'}

# directory = Path(r"E:\app_data\dropbox_13f_files\processed_tables")
big_df = big_df.astype(pd_dtypes)
big_df.info()
# big_df.to_parquet(Path.joinpath(directory, 'value_total_676.parquet'), index=False)

In [221]:
# big_df2 = pd.read_parquet(Path.joinpath(directory, "value_total_676.parquet"))
dont_have_value_in_txt = ['1026710', '1011659']
fixed_value_cik = ['1008895' , '1002152', '1006435', '1011659', '1026710', '1033427', '1033505' , '1035463', '1037389']
(big_df2.query("value_total.isna() & \
                entry_total.notna() & \
                address.str.contains('|'.join(@fixed_value_cik))==False")
 
         .drop_duplicates(subset=['address'])['address'])
                # address.str.contains('1033505') \
# df.query("cusip == '066050105'")
# value_total
# matched_strings
# [s for s in matched_strings if not re.search(r'([^\d]{3,})', s).group(0)]
# [s for s in matched_strings if not re.search(r'([^\d]{3,})', s)]

2391192     1040197/0001040197-00-500003.txt
2391339     1040197/0001040197-00-500005.txt
2391474     1040197/0001040197-00-500006.txt
2391602     1040197/0001040197-01-500010.txt
2399970     1040197/0001040197-99-000002.txt
                          ...               
15266715     947822/0000905718-00-000376.txt
15266736     947822/0000905718-01-000053.txt
15267565     947822/0000905718-99-000320.txt
15267589     947822/0000905718-99-000429.txt
15267616     947822/0000905718-99-000525.txt
Name: address, Length: 704, dtype: category
Categories (41084, object): ['1000097/0000919574-00-000478.txt', '1000097/0000919574-00-000575.txt', '1000097/0000919574-00-000601.txt', '1000097/0000919574-00-000736.txt', ..., '98758/0001193125-12-351360.txt', '98758/0001193125-12-462906.txt', '98758/0001193125-13-060787.txt', '98758/0001193125-13-213931.txt']

In [220]:
files = filings_676_txt.rglob("1037389/*.txt")  # -03-000131
# l = ('1008895/0001008895-06-000002.txt',
#  '1008895/0001008895-06-000009.txt',
#  '1008895/0001008895-06-000011.txt')

for file in files:
    if "1037389/"+file.name in l:
    
        with open(file, 'r') as f:
            # Read the contents of the file
            text = f.read()
            # print(file)
        # code to find the 2nd occurence of the substring in the text and 
        # substitute only this 2nd ocurrence with something else 
        
        # entry_total = re.compile(r"(Value \(x1000\) Total)").finditer(text)
        # matches =  [match.group(0) for match in entry_total]
        # # print(matches)
        # if len(matches) > 1:  
        #     where = [m.start() for m in re.finditer("Value (x1000) Total", text)][1-1]
        #     before = text[:where]
        #     after = text[where:]
        #     after = after.replace("Value (x1000) Total", "(x1000) Value Total", 1)
        #     updated_text = before + after
        #     print(updated_text)
        
############################## Value \(x $1000\) Total
        entry_total = re.compile(r"(Value \(x \$1000\) Total)").finditer(text)
        matches =  [match.group(0) for match in entry_total]
        # print(matches)
        if matches:  
            updated_text = text.replace("Value (x $1000) Total", "(x $1000) Value Total", 1)

            # print(updated_text[:10000])

        # Write the file out again
        with open(file, 'w') as file:
            file.write(updated_text)
    
    

In [214]:
files = list(filings_676_txt.rglob("1037389/*.txt"))
for file in files:
    if "1037389/"+file.name in l:
        print(True)


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [153]:
"1008895/"+files.name, l[0]


('1008895/0001008895-00-000001.txt', '1008895/0001008895-06-000002.txt')

In [196]:
matches

[]

In [ ]:
txt = """
AB,CD,EF, JJ
foo,20160101,a,23
foo,20160102,a,34
foo,20160103,a,56
"""
lambda txt: len(txt) <= 10
# data = pl.read_csv(io.StringIO(txt))
# data = data.

In [52]:
# different pandas dtypes

dtypes =    {
        "ID": str,
        "accessionNumber": str,
        "cikManager": "Int64",
        "periodOfReport": "datetime64[ns]",
        "report_Quarter": "Int64",
        "report_Year": "Int64",
        "submissionType": str,
        "isAmendment": bool,
        "amendmentType": str,
        "filedAsOfDate": "datetime64[ns]",
        "entryTotal": "Int64",
        "valueTotal": "float64",
        "cusip": str,
        "nameOfIssuer": str,
        "titleOfClass": str,
        "sharesValue": "float64",
        "sharesHeldAtEndOfQtr": "Int64",
        "securityType": str,
        "putCall": str,
        "xml_flag": str,
        "created_at": "datetime64[ns]",
        "edgar_path": str,
    }